# Task 05 — Descriptive Stats & LLM Comparison (NBA 2022–23)

**Dataset:** `Nba_data.csv` (preferred)

In [ ]:
import pandas as pd, numpy as np, os, matplotlib.pyplot as plt
pd.set_option("display.float_format", lambda x: f"{x:0.3f}")

In [ ]:
# Data load preference:
# 1) Nba_data.csv (repo root)
# 2) data/nba_players_2022_23.csv
# 3) nba_2022_23_subset.csv (fallback included)

candidates = [
    "Nba_data.csv",
    "data/nba_players_2022_23.csv",
    "/mnt/data/nba_2022_23_subset.csv",
    "nba_2022_23_subset.csv",
]

source = None
for p in candidates:
    if os.path.exists(p):
        source = p
        break

if source is None:
    raise FileNotFoundError("No dataset found. Please add Nba_data.csv or data/nba_players_2022_23.csv.")

df = pd.read_csv(source)
print("Loaded:", source)
df.head()

## Clean & describe

In [ ]:
num_cols = ["PTS","REB","AST","STL","BLK","FG%","3P%","FT%"]
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")
for c in ["FG%","3P%","FT%"]:
    if c in df.columns and df[c].max(skipna=True) > 1.0:
        df[c] = df[c] / 100.0

df.describe(include='all')

## Q1: Top scorer

In [ ]:
top_scorer = df.loc[df['PTS'].idxmax()]
{'player': top_scorer['Player'], 'PTS': float(top_scorer['PTS'])}

## Q2: Best all-around (PTS+REB+AST)

In [ ]:
df['ALL_AROUND'] = df[['PTS','REB','AST']].sum(axis=1, skipna=True)
best_all = df.loc[df['ALL_AROUND'].idxmax()]
{'player': best_all['Player'], 'ALL_AROUND': float(best_all['ALL_AROUND'])}

## Q3: “Most complete” (z-score composite across available stats)

In [ ]:
from scipy.stats import zscore
features = ["PTS","REB","AST","STL","BLK","FG%","3P%","FT%"]
avail = [c for c in features if c in df.columns]
dfc = df.copy()
for c in avail:
    dfc[f"z_{c}"] = (dfc[c] - dfc[c].mean(skipna=True))/dfc[c].std(skipna=True)
zcols = [f"z_{c}" for c in avail]
dfc["COMPOSITE"] = dfc[zcols].mean(axis=1, skipna=True)
q3_table = dfc[["Player","COMPOSITE"] + avail].sort_values("COMPOSITE", ascending=False)
q3_table.head(10)

## Q4: Best shooter (leaders by FG%, 3P%, FT% + composite)

In [ ]:
sh = [c for c in ["FG%","3P%","FT%"] if c in df.columns]
leaders = {c: df.loc[df[c].idxmax(), 'Player'] for c in sh}
tmp = df.copy()
for c in sh:
    tmp[f"z_{c}"] = (tmp[c]-tmp[c].mean(skipna=True))/tmp[c].std(skipna=True)
z = [f"z_{c}" for c in sh]
tmp["SHOOTER_SCORE"] = tmp[z].mean(axis=1, skipna=True)
best_overall = tmp.loc[tmp["SHOOTER_SCORE"].idxmax(), 'Player']
{'leaders_by_stat': leaders, 'best_overall_composite': best_overall}

## Q5: Count players ≥25 PPG

In [ ]:
int((df['PTS'] >= 25).sum())

## Q6: Count players ≥8 AST

In [ ]:
int((df['AST'] >= 8).sum())

## Visuals

In [ ]:
# Top 10 scorers
top10 = df.sort_values("PTS", ascending=False).head(10)
plt.figure()
plt.bar(top10["Player"], top10["PTS"])
plt.xticks(rotation=45, ha="right")
plt.title("Top 10 — Points per Game")
plt.ylabel("PPG")
plt.tight_layout(); plt.show()

In [ ]:
# Histogram: Assists
plt.figure()
plt.hist(df["AST"].dropna(), bins=8)
plt.title("Distribution of Assists per Game")
plt.xlabel("AST"); plt.ylabel("Count")
plt.tight_layout(); plt.show()

In [ ]:
# Scatter: PTS vs AST
plt.figure()
mask = df["PTS"].notna() & df["AST"].notna()
plt.scatter(df.loc[mask,"PTS"], df.loc[mask,"AST"])
plt.title("Points vs Assists"); plt.xlabel("PTS"); plt.ylabel("AST")
plt.tight_layout(); plt.show()

## Save outputs

In [ ]:
q3_table.to_csv("q3_composite_table.csv", index=False)
summary = {
    "top_scorer_player": df.loc[df['PTS'].idxmax(), 'Player'],
    "best_all_around_player": df.loc[df['ALL_AROUND'].idxmax(), 'Player'],
    "count_25ppg": int((df['PTS'] >= 25).sum()),
    "count_8ast": int((df['AST'] >= 8).sum())
}
import json
with open("summary_answers.json", "w") as f:
    json.dump(summary, f, indent=2)
summary